# Google Analytics Customer Revenue Prediction

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

Define a function to load the CSV files

In [ ]:
%%time
# https://www.kaggle.com/julian3833/1-quick-start-read-csv-and-flatten-json-fields/notebook
def load_df(csv_path='../input/ga-customer-revenue-prediction/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    json = __import__('json')
    json_normalize = pd.io.json.json_normalize
    df = pd.read_csv(
        csv_path,
        converters={ column: json.loads for column in JSON_COLUMNS },
        dtype={ 'fullVisitorId': 'str', 'visitId': 'str' },
        nrows=nrows
    )
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f'{column}.{subcolumn}' for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f'Loaded {csv_path}. Shape: {df.shape}')
    return df

# df_train = load_df()
# df_test = load_df('../input/ga-customer-revenue-prediction/test.csv')

In [ ]:
# %%time
# df_train.to_csv('train-flattened.csv', index=False)
# df_test.to_csv('test-flattened.csv', index=False)

In [ ]:
df_train = pd.read_csv('../input/ga-store-customer-revenue/train-flattened.csv', dtype={ 'fullVisitorId': str, 'visitId': str, 'trafficSource.campaignCode': str },)
df_test = pd.read_csv('../input/ga-store-customer-revenue/test-flattened.csv', dtype={ 'fullVisitorId': str, 'visitId': str },)

## Target variable
`totals.transactionRevenue`
1. There are 903653 records in the train set, 804684 in the test set
2. Only 11515 records resulted in revenue, the remaining 892138 records had zero revenue.
3. Of these, only 9996 visitors (out of 714167 unique visitors) had non-zero revenue. This translates to: only 1.4% of visitors bought something.
4. From 2016-08 to 2017-08, GStore in this dataset had \$1,540,071  in revenue, or \$4,219 per day. With a min spend of \$0.01, max of \$23,129, mean of \$133, and median of \$49

In [ ]:
df_train['totals.transactionRevenue'] = df_train['totals.transactionRevenue'].astype(float)
df_train['totals.transactionRevenue'].notnull().sum()
np.sum(df_train.groupby('fullVisitorId')['totals.transactionRevenue'].sum() > 0)
df_train['totals.transactionRevenue'].sum() / 1000000
(df_train['totals.transactionRevenue'][df_train['totals.transactionRevenue'] > 0] / 1000000).agg(['min', 'max', 'mean', 'median'])

## Predictors
1. There are 54 predictos in the train set and 52 in the test set.
2. Remove predictors not present in the test set
3. Remove predictors where values are the same for all records
4. Plot predictors vs number or records, vs number of records with revenue, vs revenue mean, vs revenue median

In [ ]:
columns = set(df_train.columns).intersection((set(df_test.columns))) # columns present in both train and test
print('Remove columns:', end=' ')
for i in df_train.columns[df_train.nunique(dropna=False) == 1].tolist() + df_test.columns[df_test.nunique(dropna=False) == 1].tolist():
    if (i in columns):
        print(i, end=', ')
        columns.remove(i)

In [ ]:
def count(var):
    tmp = pd.concat([
        df_train[var].value_counts(),
        df_train[df_train['totals.transactionRevenue'] > 0][var].value_counts(),
        df_test[var].value_counts(),
    ], keys=['train', 'train with revenue', 'test'], axis=1, sort=False)
    tmp['train %'] = np.round(tmp['train'] / tmp['train'].sum() * 100, 1)
    tmp['train with revenue %'] = np.round(tmp['train with revenue'] / tmp['train with revenue'].sum() * 100, 1)
    tmp['test %'] = np.round(tmp['test'] / tmp['test'].sum() * 100, 1)
    print(tmp.shape)
    return(tmp)

### channelGrouping
- There are 8 unique levels
- The majority is "Organic Search"

In [ ]:
count('channelGrouping')

### Date & visitStartTime
- convert to datetime
- Most date and visitStartTime are similar, some converstion from epoch to datetime in visitStartTime resulted in a diference of -1 days, likely due to not using the correct timezone. But this doesn't really matter.
- Will remove the date column as most information can be extracted from visitStartTime

In [ ]:
df_train['date'] = pd.to_datetime(df_train['date'], format='%Y%m%d')
df_train['visitStartTime'] = pd.to_datetime(df_train['visitStartTime'], unit='s')
df_test['date'] = pd.to_datetime(df_train['date'], format='%Y%m%d')
df_test['visitStartTime'] = pd.to_datetime(df_train['visitStartTime'], unit='s')
(df_train['date'].dt.date - df_train['visitStartTime'].dt.date).value_counts() # this show most date and visitStartTime are the same
columns.remove('date') # remove the date column

### device.browser
- There are 192 of them in both train and test
- The most populare is Chrome (68.7%), followed by Safari, then Firefox.
- Some of them are present only either train or test set and the counts are very low.

In [ ]:
count('device.browser')

### device.deviceCategory
- Desktop takes up ~ 3 quarters of the train set, about 1/3rd in the test set
- Desktop contributed more than 90% of the total revenue

In [ ]:
count('device.deviceCategory')

### device.operatingSystem
- 24 levels in total
- 6 of them: Windows, macintosh, Android, iOS, Linux and Chrome constitue to more than 99% of the market

In [ ]:
count('device.operatingSystem')

   ### geoNetwork.continent, geoNetwork.subContinent
   - 5 continents + unset
   - 23 subContinents
   - America constituted to nearly half of the traffic and contributed to almost more than 95% of the total revenues
   - Europe and Asia have also nearly half of the traffic, but their combined revenues are 2% of the total revenues

In [ ]:
count('geoNetwork.continent')
count('geoNetwork.subContinent')

### geoNetwork.country, geoNetwork.region, geoNetwork.city, geoNetwork.metro
- 228 cities, 483 regions, 956 cities, 123 metros
- US has 40% traffic, but ~95% revenue
- Region: highest traffic from "not available" (56%), followed by California (12%). But the "not available" only had ~40% revenue, while California had ~29% revenue. Interestingly, New York also has high revenue (13%) relative to its traffic (2.9%).
- City identified as "not available" has the highest traffic (56%) with ~ 39% revenue. The next in line is Mountain View with 4.5% traffic, but ~11% revenue.
- Similary, metro of "not available" and "not set" have the highest combined traffic (~78%), but only ~42% revenue. Whereas San Francisco-Oakland-San Jose and New York have ~14% traffic but 39% revennue.

### geoNetwork.networkDomain
- 41982 network domains
- 40%  are "not set" and "unknown", but they contributed to ~61% of the total revenue

In [ ]:
count('geoNetwork.networkDomain')
count('geoNetwork.country')
count('geoNetwork.region')
count('geoNetwork.city')
count('geoNetwork.metro')

### trafficSource.adContent, trafficSource.adwordsClickInfo.adNetworkType, trafficSource.adwordsClickInfo.gclId, trafficSource.adwordsClickInfo.isVideoAd, trafficSource.adwordsClickInfo.page, trafficSource.adwordsClickInfo.slot, trafficSource.campaign, trafficSource.isTrueDirect, trafficSource.keyword, trafficSource.medium, trafficSource.referralPath, trafficSource.source
- 76 ad content, with Google Merchandise Collection having ~47% traffic and 82% revenue. Google Online Store has 11% traffic but ~3% revenue.

In [ ]:
count('trafficSource.adContent')

### Various Id
 - 902755 sessionId
 - 886303 visitId
 - 714167  fullVisitorId

## Plots

In [ ]:
def make_button(var, title = '', max_row = 10, df = df_train, target_var = 'totals.transactionRevenue'):
    # create data for the 'updatemenus' used by plotly
    # agg data from var ~ target_vaar into size, count, mean, median
    # return dict()
    tmp = df.groupby(var)[target_var].agg(['size', 'sum', 'mean', 'median'])
    tmp = tmp.sort_values('size', ascending=False)[:max_row][::-1] # by defaul, take only the top 10 rows ordered by size
    tmp = {
        'x': [tmp['size'].values, tmp['sum'].values , tmp['mean'].values, tmp['median'].values],
        'y': [[str(i) for i in tmp.index.tolist()]] * 4, # str(i) to convert all to string, because some of the indexes are True, False
    }
    title = title or var
    return dict(args=[tmp, { 'title': title }], label=title, method='update') 

# plotting
## data
tmp = make_button('device.deviceCategory', 'Device Category')
x = tmp['args'][0]['x']
y = tmp['args'][0]['y'][0]

## trace
traces = [None] * 4
traces[0] = (go.Bar(x=x[0], y=y, orientation='h'))
traces[1] = (go.Bar(x=x[1], y=y, orientation='h'))
traces[2] = (go.Bar(x=x[3], y=y, orientation='h', name='Median')) # median goes first, the resulting bar graph will place median at the bottom
traces[3] = (go.Bar(x=x[2], y=y, orientation='h', name='Mean'))

## fig, subplot
fig = __import__('plotly').tools.make_subplots(1, 3, subplot_titles=['Number of record', 'Total revenue', 'Mean & Median'])
for i in range(3): fig.append_trace(traces[i], 1, i + 1)
fig.append_trace(traces[-1], 1, 3)

## fig, layout
fig.layout.title = tmp['args'][1]['title']
fig.layout.showlegend = False
fig.layout.updatemenus = list([
    dict(
        buttons=[make_button(i) for i in [
            'device.deviceCategory', 'device.operatingSystem', 'device.browser', 'device.isMobile',
            'geoNetwork.continent', 'geoNetwork.subContinent', 'geoNetwork.country', 'geoNetwork.region', 'geoNetwork.metro', 'geoNetwork.city', 'geoNetwork.networkDomain',
            'trafficSource.source', 'trafficSource.medium', 'trafficSource.keyword', 'trafficSource.campaign', 'trafficSource.referralPath', 'trafficSource.isTrueDirect',
            'trafficSource.adContent', 'trafficSource.adwordsClickInfo.adNetworkType', 'trafficSource.adwordsClickInfo.gclId',
        ]],
        direction = 'down',
        showactive = True,
        x = 0,
        xanchor = 'left',
        y = 1.25,
        yanchor = 'top' 
    ),
])

## now plot
py.iplot(fig)

- **Device category**: desktop has a higher total revenue compared to mobile & tablet proporrtional to their traffic.
- **Operating systerm**: Macintosh has a higher revenues even though it has lower traffic compared to Windoes. Chrome OS also has a high revenue relative to its traffic.
- **Browser**: Chrome has the higest traffic and revenue. Safari has the 2nd highest traffic, but its revenue is comparatively lower based on its traffic.
- **Is mobile**: Mobile traffic is just a litte bit over 1/3rd of non-mobile, but its revenue is only 1/10th of non-mobile.

- **Continent & subContinent**: Americas has proportinally higher revenues compared to Asia & Europe. Revenues came mostly from Northern America even though other continents have high traffic.
- **Country**: again most revenues came from US.
- **Region**: revenues came mostly from "not vailable in demo dataset", followed by California. Interestingly, New York, Texas, Washington & Illinois all have high revenues relative to their traffic.
- **City**: again revenues came mostly from "not avilable in demo dataset". New York, Mountain View, San Francisco, Los Angeles have high revenues relative to their traffic.
- **Metro**: similar patterns to City.
- **Network domain**: revneues come predominantly from "(not set)". The unknown.unknow has a relatively high traffic, but low revenues.

- **Traffic source**: mall.googleplex.com generated most revenues, given that it has lower traffic compared to the other. Youtube did poorly given it had high traffic.
- **Medium**: referral generated a higher revenue than the other. It's understandable. Affiliate generated a much less revenue based on its traffic.
- **Keyword, campain**: "(not provided)", "(not set)" have the highest revenue. Nothing much can be said, many of them are "code".
- **Ad content**: Google Merchandise Collection has the highest revenue. Google online store doesn't seem to fare well given its traffic.
- **gclId**: Google Click Identifier, nothing much can be said.

In [ ]:
df_train.visitStartTime